In [18]:
import pandas as pd
import numpy as np 
from datetime import datetime
import re

In [19]:
path = ".csv" # change this code 
total_que_num = 100

In [20]:
df_m = pd.read_csv(path,header=0)
df_m

,タイムスタンプ,ユーザー名,問題番号。(半角数字のみ。問や他の記号を入れない。枝問は小数点を使用),問題文 。 (問題番号をここに入れないでください。),選択肢。( 選択肢ごとに、改行をお願いします。) Ex.a) リネゾリドは、...をきたす。(改行) b) バンコマイシンは...をきたす。 （改行）,"回答の選択肢。自由選択の場合は""x""のみ書いてください。Ex. 1)a 2)b.c 3)b,d,e 4)d,e (問題用紙の通り入力、改行不要 )",何か気づいたことがあれば入力してください。
0,2020/12/18 9:03:46 午後 GMT+9,gmkw53111.med@gmail.com,1,test,NaN,NaN,NaN
1,2020/12/18 9:04:47 午後 GMT+9,gmkw53111.med@gmail.com,100,y=x-3,a\naa\naaa,"1,2,3 3,4,5",わかりません


In [21]:
df_m = pd.read_csv(path,header=0)
df_m.columns = ["time","e_mail","que_num","question","choice","ans_choice","comment"]
df = df_m
df

,time,e_mail,que_num,question,choice,ans_choice,comment
0,2020/12/18 9:03:46 午後 GMT+9,gmkw53111.med@gmail.com,1,test,NaN,NaN,NaN
1,2020/12/18 9:04:47 午後 GMT+9,gmkw53111.med@gmail.com,100,y=x-3,a\naa\naaa,"1,2,3 3,4,5",わかりません


In [22]:
df_sort = df.sort_values(by =  "que_num",ascending=True).reset_index(drop=True)\
    .dropna(subset=["que_num"])
df_sort

,time,e_mail,que_num,question,choice,ans_choice,comment
0,2020/12/18 9:03:46 午後 GMT+9,gmkw53111.med@gmail.com,1,test,NaN,NaN,NaN
1,2020/12/18 9:04:47 午後 GMT+9,gmkw53111.med@gmail.com,100,y=x-3,a\naa\naaa,"1,2,3 3,4,5",わかりません


In [23]:
regex = re.compile(r'^.{,3}[×xX].{,3}$')
regex_hatena = re.compile(r'^.{,3}\?.{,3}$')
regex_hate = re.compile(r'.*\?.*')

bool_que = [ False for i in range(total_que_num + 1)]
bool_que_sent = [ False for i in range(total_que_num + 1)]
bool_choice = [ False for i in range(total_que_num + 1)]
bool_ans = [ False for i in range(total_que_num + 1)]
bool_que = [ False for i in range(total_que_num + 1)]

res = ""
for i in range(df_sort.shape[0]):
    index = i
    que_n = int(df_sort["que_num"][index])
    question =  str(df_sort["question"][index])
    choice = str(df_sort["choice"][index])
    ans_choice = str(df_sort["ans_choice"][index])
    comment = str(df_sort["comment"][index])
    e_mail =  str(df_sort["e_mail"][index])
    
    bool_que[que_n] = True 
    if len(regex_hatena.findall(question)) > 0:
        bool_que_sent[que_n] = True
    if len(regex_hate.findall(choice)) > 0:
        bool_choice[que_n] = True
    if len(regex_hate.findall(ans_choice)) > 0:
        bool_ans[que_n] = True
    
    
    if len(regex.findall(ans_choice)) == 0:
        res = res + "問" + str(que_n) + ". " + question + "\n" + choice  + "\n" + ans_choice + "\n" + "コメント：" + comment + "\n" + "担当連絡先：" + e_mail + "\n\n"
    else:
        res = res + "問" + str(que_n) + ". " + question + "\n" + choice  + "\n" + "コメント：" + comment + "\n" + "担当連絡先：" + e_mail + "\n\n"

In [24]:
lack_num = "【提出されてない、もしくは、一部わからない問題】" + "\n"
for i in range(1,total_que_num+1):
    if not bool_que[i]:
        lack_num = lack_num + str(i) + ", "
    else:
        flag = ""
        if  bool_que_sent[i]:
            flag = flag + "(問題文)"
            
        if  bool_choice[i]:
            flag = flag + "(選択肢)"
        if  bool_ans[i]:
            flag = flag + "(回答の選択肢)"
        if len(flag) > 0:
            lack_num = lack_num + str(i) + flag + ", "
            
que_tx = lack_num + "\n\n\n\n" + res         

In [25]:
output_name = "output_" + path + "_" + ".txt"
with open(output_name,"w") as f:
    f.writelines(que_tx)